In [1]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
! pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 3.3 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149857 sha256=b7ba2702704b9deab0e7d6f301884974b5d138c24aac1c07d62904703dc263fc
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [55]:
import gensim
import pandas as pd
import numpy as np

In [29]:
df = pd.read_csv("/content/Movie_title.csv")

In [30]:
df.head()

,movie_title,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Intolerance: Love's Struggle Throughout the Ages,NaN,NaN,NaN,
1,Over the Hill to the Poorhouse,NaN,NaN,NaN,
2,The Big Parade,NaN,NaN,NaN,
3,Metropolis,NaN,NaN,NaN,
4,Pandora's Box,NaN,NaN,NaN,


In [31]:
df.movie_title[0]

"Intolerance: Love's Struggle Throughout the Ages"

In [32]:
gensim.utils.simple_preprocess("Intolerance: Love's Struggle Throughout the Ages")

['intolerance', 'love', 'struggle', 'throughout', 'the', 'ages']

In [33]:
movie_title = df.movie_title.apply(gensim.utils.simple_preprocess)
movie_title

0      [intolerance, love, struggle, throughout, the,...
1                  [over, the, hill, to, the, poorhouse]
2                                     [the, big, parade]
3                                           [metropolis]
4                                         [pandora, box]
                             ...                        
595    [marilyn, hotchkiss, ballroom, dancing, and, c...
596                                       [metropolitan]
597                                  [midnight, cabaret]
598                                             [nikita]
599                                        [dark, angel]
Name: movie_title, Length: 600, dtype: object

In [53]:
model = gensim.models.Word2Vec(
    window = 3,
    min_count = 1,
    workers = 4
)

In [54]:
model.build_vocab(movie_title, progress_per = 50)

In [36]:
model.epochs 

5

In [37]:
model.corpus_count

600

In [38]:
model.train(movie_title,total_examples=model.corpus_count,epochs=model.epochs )

(6726, 8445)

In [39]:
model.save("/content/model")

In [41]:
model.wv.most_similar("metropolis")

[('titanic', 0.29598963260650635),
 ('nun', 0.2859593629837036),
 ('ted', 0.2804208993911743),
 ('em', 0.2418500781059265),
 ('ask', 0.23835575580596924),
 ('flash', 0.22740423679351807),
 ('pierrot', 0.21153181791305542),
 ('warriors', 0.21091249585151672),
 ('fog', 0.21053524315357208),
 ('quo', 0.20930296182632446)]

In [46]:
model.wv.similarity(w1="love",w2="struggle")

0.05605492

In [50]:
movie_title.to_csv("titlewv.csv")

In [57]:
vocab, vectors = model.wv.vocab, model.wv.vectors

# get node name and embedding vector index.
name_index = np.array([(v[0], v[1].index) for v in vocab.items()])

# init dataframe using embedding vectors and set index as node name
df =  pd.DataFrame(vectors[name_index[:,1].astype(int)])
df.index = name_index[:, 0]
df.to_csv("movie_title_embedding.csv")

In [58]:
# Save model
filename = 'stored_model.wv' # Can be any arbitrary filename
model.save(filename) 

# Reload model
retrieved_model = gensim.models.Word2Vec.load(filename)